## Ring-hydroxylating Dioxygenases

In [51]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import re
from prince import CA, MCA
import matplotlib.cm as cm
from Bio.SeqUtils import seq3
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import OPTICS
import networkx as nx
from itertools import combinations
from sklearn.manifold import MDS
from scipy.cluster.hierarchy import to_tree
from operator import itemgetter
import fastcluster
import scipy.cluster.hierarchy as hierarchy
from scipy.cluster.hierarchy import fcluster
from scipy.spatial.distance import squareform
from sklearn.preprocessing import MinMaxScaler

# Download necessary resources from NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /Users/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/lucas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/lucas/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# # # EXAMPLE of how to merge data frames (alignment + metadata)
# # Join the dataframes based on the 'ID' column
# if len(df.columns) == len(col_idx):
#     if (df.columns == col_idx).all():
#         df = pd.merge(
#             pd.merge(
#                 pd.read_csv('data.tsv', delimiter='\t'),
#                 pd.DataFrame(
#                     {
#                         'Entry':[headers[idx].split('/')[0].split('_')[0] for idx in df.index],
#                         'Index':list(df.index)
#                     }
#                 ),
#                 on='Entry'
#             ),
#             df,
#             left_on='Index',
#             right_index=True,
#             how='inner'
#         ).copy()
# print(df.info)

def generate_wordcloud(df):
    """
    :return:
    """
    # Extract the substrate and enzyme names using regular expressions
    matches = df['Protein names'].str.extract(r'(.+?) ([\w\-,]+ase)', flags=re.IGNORECASE)
    # String normalization pipeline
    df['Substrate'] = matches[0] \
        .fillna('') \
        .apply(lambda x: '/'.join(re.findall(r'\b(\w+(?:ene|ine|ate|yl))\b', x, flags=re.IGNORECASE))) \
        .apply(lambda x: x.lower())
    df['Enzyme'] = matches[1] \
        .fillna('') \
        .apply(lambda x: x.split('-')[-1] if '-' in x else x) \
        .apply(lambda x: x.lower())
    df['Label'] = df['Substrate'].str.cat(df['Enzyme'], sep=' ').str.strip()
    df = df.copy()
    # Plot the word cloud
    plt.figure(figsize=(10, 6))
    plt.imshow(
        WordCloud(
            width=800,
            height=400,
            background_color='white'
        ).generate(
            ' '.join(
                sorted(
                    set([string for string in df.Label.values.tolist() if len(string) > 0])
                )
            )
        ),
        interpolation='bilinear'
    )
    plt.axis('off')
    plt.show()
    return None

def parse_msa_file(msa_file, msa_format="fasta"):
    """
    :return:
    """
    headers, sequences = [], []
    for record in SeqIO.parse(msa_file, msa_format).records:
        headers.append(record.id)
        sequences.append(record.seq)
    seq_array = np.array(sequences)
    return seq_array, headers

def clean_msa(df, threshold=.9, plot=False):
    """
    :return:
    """
    df.replace(
        ['-', *[chr(i) for i in range(ord('a'), ord('z')+1)]],
        np.nan,
        inplace=True
    )
    min_rows = int(threshold * df.shape[0])
    # Remove columns with NaN values above the threshold
    df.dropna(thresh=min_rows, axis=1, inplace=True)
    min_cols = int(threshold * df.shape[1])
    # Remove rows with NaN values above the threshold
    df.dropna(thresh=min_cols, axis=0, inplace=True)
    if plot:
        # Plot the heatmap
        sns.heatmap(df.isna().astype(int), cmap='binary', xticklabels=False, yticklabels=False, cbar=False)
        # Show the plot
        plt.show()
    return df

def shrink(data):
    """
    :return:
    """
    # Perform MCA
    mca = MCA()
    mca.fit(data)
    return mca

def get_clusters(mca_object, plot=False):
    """
    :return:
    """
    # Access the results
    coordinates = mca_object.transform(data)
    # Convert the coordinates to a NumPy array
    coordinates = np.array(coordinates)
    # Define a range of potential number of clusters to evaluate
    min_clusters = 3
    max_clusters = 10
    # Perform clustering for different number of clusters and compute silhouette scores
    silhouette_scores = []
    for k in range(min_clusters, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, n_init=10)  # Set n_init explicitly
        kmeans.fit(coordinates)
        labels = kmeans.labels_
        score = silhouette_score(coordinates, labels)
        silhouette_scores.append(score)
    # Find the best number of clusters based on the highest silhouette score
    best_num_clusters = np.argmax(silhouette_scores) + min_clusters
    # Perform clustering with the best number of clusters
    kmeans = KMeans(n_clusters=best_num_clusters, n_init=10)  # Set n_init explicitly
    kmeans.fit(coordinates)
    kmeans_cluster_labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_
    if plot:
        if len(data) <= 5000:
            # Plot together the scatter plot of both sequences and residues overlaid
            mca.plot(
                data,
                x_component=0,
                y_component=1
            )
        # Plot the scatter plot colored by clusters
        plt.scatter(coordinates[:, 0], coordinates[:, 1], c=kmeans_cluster_labels, cmap='viridis')
        plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1], c='red', marker='x', label='Cluster Centers')
        plt.xlabel('Dimension 1')
        plt.ylabel('Dimension 2')
        plt.title('Scatter Plot - Clusters')
        plt.legend()
        plt.show()
    return kmeans_cluster_labels

def henikoff(data):
    """
    :return:
    """
    data_array = data.to_numpy()  # Convert DataFrame to NumPy array
    size, length = data_array.shape
    weights = []
    for seq_index in range(size):
        row = data_array[seq_index, :]
        unique_vals, counts = np.unique(row, return_counts=True)
        k = len(unique_vals)
        matrix_row = 1. / (k * counts)
        weights.append(np.sum(matrix_row) / length)
    return pd.Series(weights, index=data.index)

def select_features(data, labels, plot=False):
    X = data
    y = pd.get_dummies(labels).astype(int) if len(
        set(labels)
    ) > 2 else labels
    # Perform one-hot encoding on the categorical features
    encoder = OneHotEncoder()
    X_encoded = encoder.fit_transform(X)
    # Get the column names for the encoded features
    encoded_feature_names = []
    for i, column in enumerate(X.columns):
        categories = encoder.categories_[i]
        for category in categories:
            feature_name = f'{column}_{category}'
            encoded_feature_names.append(feature_name)
    # Convert X_encoded to DataFrame
    X_encoded_df = pd.DataFrame.sparse.from_spmatrix(X_encoded, columns=encoded_feature_names)
    # Create and train the Random Forest classifier
    rf = RandomForestClassifier()
    rf.fit(X_encoded_df, y)
    # Feature selection
    feature_selector = SelectFromModel(rf, threshold='median')
    feature_selector.fit_transform(X_encoded_df, y)
    selected_feature_indices = feature_selector.get_support(indices=True)
    selected_features = X_encoded_df.columns[selected_feature_indices]
    # Calculate feature importances for original columns
    sorted_importance = pd.DataFrame(
        {
            'Residues': selected_features,
            'Importance': rf.feature_importances_[selected_feature_indices],
            'Columns': map(lambda x: int(x.split('_')[0]), selected_features)
        }
    )[['Columns', 'Importance']].groupby('Columns').sum()['Importance'].sort_values(ascending=False)
    sorted_features = sorted_importance.index
    if plot:
        fig, ax1 = plt.subplots(figsize=(16, 4))
        # Bar chart of percentage importance
        xvalues = range(len(sorted_features))
        ax1.bar(xvalues, sorted_importance, color='b')
        ax1.set_ylabel('Summed Importance')
        ax1.tick_params(axis='y')
        # Line chart of cumulative percentage importance
        ax2 = ax1.twinx()
        ax2.plot(xvalues, np.cumsum(sorted_importance) / np.sum(sorted_importance), color='r', marker='.')
        ax2.set_ylabel('Cumulative Importance')
        ax2.tick_params(axis='y')
        # Rotate x-axis labels
        plt.xticks(xvalues, sorted_features)
        plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=90)
        plt.show()
    return selected_features, sorted_importance

def get_labels(data, plot=False):
    """
    :return:
    """
    # Perform MCA
    mca_object = shrink(data)
    # Perform feature selection on data
    selected_features, sorted_importance = select_features(data, labels=get_clusters(mca_object))
    # Calculate cumulative sum of importance
    cumulative_importance = np.cumsum(sorted_importance) / np.sum(sorted_importance)
    # Find the index where cumulative importance exceeds or equals 0.9
    index = np.where(cumulative_importance >= 0.9)[0][0]
    # Get the values from sorted_features up to the index
    selected_columns = sorted_importance.index[:index+1].values
    # Filter the selected features to get most important residues
    selected_residues = [x for x in selected_features if int(x.split('_')[0]) in selected_columns]
    df_res = mca_object.column_coordinates(data[selected_columns]).loc[selected_residues]
    # Get sequence weights through Henikoff & Henikoff algorithm
    weights = henikoff(data[selected_columns])
    # Create an empty graph
    G = nx.Graph()
    for idx in df_res.index:
        col, aa = idx.split('_')
        col = int(col)
        rows = data[selected_columns].index[data[selected_columns][col] == aa].tolist()
        # Filter and sum values based on valid indices
        p = weights.iloc[[i for i in rows if i < len(weights)]].sum()
        # Add a node with attributes
        G.add_node(
            f'{seq3(aa)}{col}',
            idx=idx,
            aa=aa,
            col=col,
            coord=(
                df_res.loc[idx,0],
                df_res.loc[idx,1]
            ),
            rows=rows,
            p=p
        )
    nodelist = sorted(G.nodes(), key=lambda x: G.nodes[x]['p'], reverse=True)
    df_res = df_res.loc[[G.nodes[u]['idx'] for u in nodelist]]
    df_res.columns = ['x_mca', 'y_mca']
    df_res = df_res.copy()
    # Generate pairwise combinations
    pairwise_comparisons = list(combinations(G.nodes, 2))
    # Add edges to graph based on pairwise calculation of Jaccard's dissimilarity (1 - similarity)
    for u, v in pairwise_comparisons:
        asymmetric_distance = set(G.nodes[u]['rows']) ^ set(G.nodes[v]['rows'])
        union = set(G.nodes[u]['rows']) | set(G.nodes[v]['rows'])
        weight = float(
            weights.iloc[[i for i in list(asymmetric_distance) if i < len(weights)]].sum()
        ) / float(
            weights.iloc[[i for i in list(union) if i < len(weights)]].sum()
        ) if G.nodes[u]['col'] != G.nodes[v]['col'] else 1.
        G.add_edge(
            u,
            v,
            weight = weight
        )
    # Generate distance matrix
    D = nx.to_numpy_array(G, nodelist=nodelist)
    # Apply OPTICS on the points
    optics = OPTICS(metric='precomputed', min_samples=3)
    optics.fit(D)
    # Retrieve cluster labels
    cluster_labels = optics.labels_
    # Find the unique values and their counts
    unique_values, counts = np.unique(cluster_labels, return_counts=True)
    # Calculate the proportion of unique values
    proportions = counts / len(cluster_labels)




    # # Need refactoring so clusters contain residue objects instead of just indices
    # unique_labels = np.unique(cluster_labels)
    # clusters = [[] for _ in unique_labels]
    # for i, cluster_label in enumerate(unique_labels):
    #     clusters[i] = np.where(cluster_labels == cluster_label)[0].tolist()
    # adhesion = []
    # for row_idx in data.index:
    #     temp = []
    #     for cluster in clusters:
    #         count = 0
    #         for cluster_idx in cluster:
    #             node_idx = nodelist[cluster_idx]
    #             if data.loc[row_idx, G.nodes[node_idx]['col']] == G.nodes[node_idx]['aa']:
    #                 count += 1
    #         temp.append(float(count)/float(len(cluster)))
    #     adhesion.append(np.array(temp))





    adhesion = np.array(adhesion)
    df_adh = pd.DataFrame(adhesion)
    df_adh.columns = [f"Cluster {label+1}" if label >= 0 else "Noise" for label in unique_labels]
    itemgetter_func = itemgetter(*data.index)
    seq_ids = itemgetter_func(headers)
    df_adh.index = seq_ids
    # Run clustermap with potential performance improvement
    Z = fastcluster.linkage(df_adh, method='ward')
    # List to store silhouette scores
    silhouette_scores = []
    # Define a range of possible values for k
    k_values = range(2, 10)
    # Calculate silhouette score for each value of k
    for k in k_values:
        labels = fcluster(Z, k, criterion='maxclust')
        silhouette_scores.append(silhouette_score(df_adh, labels))
    # Find the index of the maximum silhouette score
    best_index = np.argmax(silhouette_scores)
    # Get the best value of k
    best_k = k_values[best_index]
    # Get the cluster labels for each sequence in the MSA
    sequence_labels = fcluster(Z, best_k, criterion='maxclust')
    if plot:
        # Plot the distance matrix
        fig, ax = plt.subplots()
        im = ax.imshow(D, cmap='viridis')
        # Add a colorbar
        cbar = ax.figure.colorbar(im, ax=ax)
        # Retrieve ordering information
        ordering = optics.ordering_
        # Perform MDS to obtain the actual points
        mds = MDS(n_components=2, dissimilarity='precomputed', normalized_stress='auto')
        points = mds.fit_transform(D)

        df_res['x_mds'] = points[:,0]
        df_res['y_mds'] = points[:,1]
        df_res['label'] = cluster_labels
        df_res = df_res.copy()
        # Plot the ordering analysis
        plt.figure(figsize=(12, 4))
        plt.bar(range(len(ordering)), ordering, width=1., color='black')
        plt.xlim([0, len(ordering)])
        plt.xlabel('Points')
        plt.ylabel('Reachability Distance')
        plt.title('Ordering Analysis')
        # Show the plots
        plt.show()
        # Create a figure with two subplots
        fig, axs = plt.subplots(1, 2, figsize=(12, 7))
        # Plot for MCA
        axs[0].set_xlabel('MCA Dimension 1', fontsize=14)
        axs[0].set_ylabel('MCA Dimension 2', fontsize=14)
        axs[0].set_title('Residues from MCA Colored by Cluster', fontsize=16)
        # Plot for MDS
        axs[1].set_xlabel('MDS Dimension 1', fontsize=14)
        axs[1].set_ylabel('MDS Dimension 2', fontsize=14)
        axs[1].set_title('Residues from MDS Colored by Cluster', fontsize=16)
        x_mca = df_res['x_mca']
        y_mca = df_res['y_mca']
        x_mds = df_res['x_mds']
        y_mds = df_res['y_mds']
        labels = df_res['label']
        unique_labels = np.unique(labels)
        color_map = cm.get_cmap('tab10', len(unique_labels))
        handles = []
        all_labels = []
        for i, cluster_label in enumerate(unique_labels):
            if cluster_label == -1:
                noise_color = 'grey'
                noise_marker = 'x'
                label = 'Noise'
            else:
                noise_color = color_map(i)
                noise_marker = 'o'
                label = f'Cluster {cluster_label+1}'
            scatter_mca = axs[0].scatter(x_mca[labels == cluster_label], y_mca[labels == cluster_label], color=noise_color, marker=noise_marker, label=label)
            scatter_mds = axs[1].scatter(x_mds[labels == cluster_label], y_mds[labels == cluster_label], color=noise_color, marker=noise_marker, label=label)
            # Collect the scatter plot handles and labels
            handles.append(scatter_mca)
            all_labels.append(label)
        # Rearrange labels to fill one row first and then the second row
        n_cols = 6  # Number of columns in the legend
        all_labels_reordered = [all_labels[i::n_cols] for i in range(n_cols)]
        all_labels_reordered = sum(all_labels_reordered, [])  # Flatten the nested list
        # Create a single legend for both subplots with reordered labels
        fig.legend(handles, all_labels_reordered, bbox_to_anchor=(0.5, 0.0), loc='upper center', borderaxespad=0, ncol=n_cols, fontsize=12)
        # Adjust the layout to accommodate the legend
        fig.tight_layout(rect=[0, 0.05, 1, 0.9])
        cbar_kws = {"orientation": "horizontal"}
        g = sns.clustermap(df_adh.drop('Noise', axis=1), col_cluster=False, yticklabels=False, cbar_pos=(.4, .9, .4, .02), cbar_kws=cbar_kws, figsize=(5, 5))
        # Show the plot
        plt.show()
    return cluster_labels, sequence_labels


def refine():
    """
    :return:
    """
    return None

def bootstrap():
    """
    :return:
    """
    return None

def get_newick(node, parent_dist, leaf_names, newick=""):
    """
    :return:
    """
    if node.is_leaf():
        return f"{leaf_names[node.id]}:{parent_dist - node.dist:.2f}{newick}"
    else:
        if len(newick) > 0:
            newick = f"):{parent_dist - node.dist:.2f}{newick}"
        else:
            newick = ");"
        newick = get_newick(node.get_left(), node.dist, leaf_names, newick)
        newick = get_newick(node.get_right(), node.dist, leaf_names, f",{newick}")
        newick = f"({newick}"
        return newick


In [ ]:
# Create data frame from metadata and generate word clouds
generate_wordcloud(pd.read_csv('data.tsv', delimiter='\t'))

In [4]:
# Create data frame from raw data and clean it
raw, headers = parse_msa_file('alignment.fasta')
msa = pd.DataFrame(raw)
df = clean_msa(msa)
# Backup original indices for further usage
row_idx, col_idx = df.index, df.columns
print(df.info)

8442 36
<bound method DataFrame.info of      182 185 186 187 200 204 205 206 216 217  ... 838 841 845 848 851 853 982   
0      A   L   Y   C   E   G   F   H   V   H  ...   T   Y   V   Y   D   E   E  \
1      K   L   Y   M   D   P   Y   H   L   H  ...   Y   F   G   Y   E   D   D   
2      K   L   T   Y   E   N   Y   H   I   H  ...   L   Y   W   K   G   E   E   
3      K   V   F   C   D   G   Y   H   A   H  ...   Y   F   L   E   G   S   E   
4      K   V   V   N   E   C   Y   H   N   H  ...   W   F   V   A   D   A   Q   
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..   
9373   K   T   F   I   E   D   Y   H   F   H  ...   F   F   Y   P   E   E   E   
9376   K   M   Y   L   D   T   V   H   T   H  ...   S   F   R   A   A   G   D   
9377   K   L   T   M   E   C   Y   H   N   H  ...   W   L   V   H   R   D   Q   
9378   K   L   V   V   D   F   Y   H   V   H  ...   V   F   V   D   N   S   Q   
9380   K   L   T   M   E   C   Y   H   N   H  ...   W   C   V   H   R

In [13]:
# Define the main data set to be used along the data flow
data = df[col_idx].drop_duplicates().fillna('-').copy()
print(data.info)

<bound method DataFrame.info of      182 185 186 187 200 204 205 206 216 217  ... 838 841 845 848 851 853 982   
0      K   L   L   I   E   C   Y   H   S   H  ...   V   Y   R   H   R   D   E  \
1      K   S   I   V   E   C   Y   H   A   H  ...   I   Y   F   L   N   K   E   
2      K   N   I   V   E   C   Y   H   A   H  ...   I   Y   F   T   N   E   E   
3      K   T   L   A   E   C   Y   H   A   H  ...   V   Y   R   H   V   D   E   
4      K   I   I   V   E   C   Y   H   A   H  ...   I   Y   F   R   N   K   E   
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..   
7218   K   M   V   V   D   G   Y   H   T   H  ...   A   V   L   I   P   G   D   
7221   K   L   Y   M   E   F   Y   H   L   H  ...   L   Y   F   P   A   P   Q   
7223   K   M   V   V   D   G   Y   H   T   H  ...   A   V   L   M   P   G   D   
7224   K   V   L   V   E   G   Y   H   T   H  ...   S   M   V   T   P   M   -   
7225   K   T   F   I   E   D   Y   H   F   H  ...   F   Y   Y   P   E   D   E

In [32]:
cluster_labels, sequence_labels = get_labels(data)

# Print the cluster labels
print(np.unique(sequence_labels))

Best k: 8
Best silhouette score: 0.4250574761267073
[1 2 3 4 5 6 7 8]


In [58]:
labels

array([8, 8, 8, ..., 3, 8, 7], dtype=int32)